In [1658]:
# importamos las librerías necesarias
import numpy as np
import math as m
import sympy as sp
from sympy import *
import cv2                      # opencv
import matplotlib.pyplot as plt # pyplot


In [1659]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

In [1660]:
import sim
import time
def connect(port):
# Establece la conexión a COPPELIA
# El port debe coincidir con el puerto de conexión en VREP  -- DALE AL PLAY !!!
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

In [1661]:
# Conectarse al servidor de COPPELIA
# *** _Hay que ejecutarlo cada vez que se reinicia la simulación ***
clientID = connect(19999)

conectado a 19999


In [1662]:
retCode,target=sim.simxGetObjectHandle(clientID,'ref_point',sim.simx_opmode_blocking)
retCode,home=sim.simxGetObjectHandle(clientID,'home',sim.simx_opmode_blocking)
retcode,rotate_joint = sim.simxGetObjectHandle(clientID,'Rotate_joint',sim.simx_opmode_blocking)
retcode,traslate_joint = sim.simxGetObjectHandle(clientID,'Traslate_joint',sim.simx_opmode_blocking)
retCode,sensorHandle=sim.simxGetObjectHandle(clientID,'Vision_sensor',sim.simx_opmode_blocking)
#retcode,traslate_joint2 = sim.simxGetObjectHandle(clientID,'Traslate_joint',sim.simx_opmode_blocking)
retCode,pos_inf=sim.simxGetObjectHandle(clientID,'pos_inf',sim.simx_opmode_blocking)
retCode,pos_sup=sim.simxGetObjectHandle(clientID,'pos_sup',sim.simx_opmode_blocking)
retCode,pos_inf_2=sim.simxGetObjectHandle(clientID,'pos_inf_2',sim.simx_opmode_blocking)
retCode,pos_sup_2=sim.simxGetObjectHandle(clientID,'pos_sup_2',sim.simx_opmode_blocking)
desplazamiento = 0
retcode, grevolute_joint = sim.simxGetObjectHandle(clientID,'GripperRevolute_joint',sim.simx_opmode_blocking)
#Gripper sensor
retcode,gripper_sensor = sim.simxGetObjectHandle(clientID,'Proximity_sensor',sim.simx_opmode_blocking)
retcode,ground_attachpoint = sim.simxGetObjectHandle(clientID,'anchor_attachpoint',sim.simx_opmode_blocking)
#Gripper attachpoint
retcode,gripper_attachpoint = sim.simxGetObjectHandle(clientID,'ROBOTIQ_85_attachPoint',sim.simx_opmode_blocking)

In [1663]:
retCode,posicion=sim.simxGetObjectPosition(clientID,target,home,sim.simx_opmode_blocking)
#retCode,posicion_original=sim.simxGetObjectPosition(clientID,sensorHandle,-1,sim.simx_opmode_blocking)
retCode,minimo=sim.simxGetObjectPosition(clientID,pos_inf,-1,sim.simx_opmode_blocking)
retCode,maximo=sim.simxGetObjectPosition(clientID,pos_sup,-1,sim.simx_opmode_blocking)
retCode,minimo2=sim.simxGetObjectPosition(clientID,pos_inf_2,-1,sim.simx_opmode_blocking)
retCode,maximo2=sim.simxGetObjectPosition(clientID,pos_sup_2,-1,sim.simx_opmode_blocking)
posicion

[6.05359673500061e-07, 0.024999618530273438, -7.152557373046875e-07]

In [1664]:
#posicion coordenadas
origen_x = posicion[0]
origen_y = posicion[1]
origen_z = posicion[2]
#dimensiones lego
lego_longitud = 0.196
lego_altura = 0.06903
#numpiezas colocadas por columna
columnas = [0,0]
#holding
holding = 0

In [1665]:
#Funcion para abrir o cerrar la pinza remotamente:
def gripper(val, parent, child):
    # función que acciona el efector final remotamente
    #val es Int con valor 0 ó 1 para desactivar o activar el actuador final. res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
    "ROBOTIQ_85", sim.sim_scripttype_childscript,"gripperLUA",[val,parent,child],[],[],"", sim.simx_opmode_blocking)
    return res
#Funcion para detectar objecto lego
def detect(sensor):
    errorCode, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector=sim.simxReadProximitySensor(clientID,sensor, sim.simx_opmode_blocking)
    # podemos calcular la distancia con  #simx_opmode_buffer
    if detectionState==True:
        sensor_val=np.linalg.norm(detectedPoint)
        print("handle objeto detectado: ", detectedObjectHandle)
        print("distacia al objeto: ",sensor_val)
        return [detectedObjectHandle, sensor_val]
    else:
        print("No ha detectado objeto")
    return 0
    

In [1666]:
#detector=cv2.SimpleBlobDetector_create() #Verion antigua !!
#keypoints=detector.detect(thresh)

In [1667]:
#imagen con los blobs y marcas
#imagen_with_keypoints=cv2.drawKeypoints(img_BGR,keypoints,np.array([]),(0,0,255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#muestra imagen con keypoints
#plt.imshow(cv2.cvtColor(imagen_with_keypoints,cv2.COLOR_BGR2RGB))

In [1668]:
#find contours
#contours,_ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
#img_contours = cv2.drawContours(img_BGR, contours, -1, (0,0,255),3)
#plt.imshow(cv2.cvtColor(img_contours,cv2.COLOR_BGR2RGB))

In [1669]:
def vision_objetos():
    #imagen del sensor camara
    retCode, resolution, image=sim.simxGetVisionSensorImage(clientID,sensorHandle,0,sim.simx_opmode_oneshot_wait)
    img=np.array(image,dtype=np.uint8)
    img.resize([resolution[1],resolution[0],3])
    img_GRAY = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, thresh = cv2.threshold(img_GRAY,150,255,cv2.THRESH_BINARY)
    
    #find contours
    contours,_ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    img_contours = cv2.drawContours(img_BGR, contours, -1, (0,0,255),3)
    
    #selección de la pieza a escoger y movernos
    xi = []
    yi = []
    ai = []
    si = []
    oi = []

    for c in contours:
        x,y,w,h = cv2.boundingRect(c)   
        if w < 256:
            x_f = ( (256- (x + w/2) ) -128)  * ((maximo[0]-minimo[0])/256)
            #x_f =  (256 - ( x + w/2 )) / 256
            print("x_f")
            print(x_f)
            #x_f = x_f*(maximo[0] - minimo[0])
            y_f = ( (256 - (y + h/2) ) - 128)  * ((maximo[1]-minimo[1])/256)

            print(x,y,w,h)
            xi.append(x_f)
            yi.append(y_f)
            ai.append(w*h)


            rotrect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rotrect)
            box = np.int0(box)
            angle = rotrect[-1]

            if angle < -45:
                angle = -(90 + angle)
            else:
                angle = -angle
            oi.append(angle)
    

    
    return xi,yi,oi,ai
    
    

In [1670]:
def seleccion(tipo,ai):
    if(tipo == 1):
        return np.argmax(ai)
    else:
        return np.argmin(ai)


In [1671]:
def buscar_pieza(pieza):
    xi,yi,oi,ai = vision_objetos()
    indice = seleccion(pieza,ai)
    return xi[indice], yi[indice], oi[indice]




In [1672]:
def vision_objeto_unico(angle):
    #imagen del sensor camara
    retCode, resolution, image=sim.simxGetVisionSensorImage(clientID,sensorHandle,0,sim.simx_opmode_oneshot_wait)
    img=np.array(image,dtype=np.uint8)
    img.resize([resolution[1],resolution[0],3])
    img_GRAY = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, thresh = cv2.threshold(img_GRAY,150,255,cv2.THRESH_BINARY)
    
    #find contours
    contours,_ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    img_contours = cv2.drawContours(img_BGR, contours, -1, (0,0,255),3)
    

    angulo = 0
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)  
        if w < 256:
            if(w <= h):
                angulo = 90 
    
    return angle + angulo
    

In [1673]:
def funcion_coger(posicion,angulo):
    
    #sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    #time.sleep(1)
    xf = posicion[0]
    yf = posicion[1]
    posicion[0] = 0
    posicion[1] = 0
    if(xf >= 0):
        for i in range(0,int(xf*1000),1):
            posicion[0] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    else:
        for i in range(0,int(xf*1000),-1):
            posicion[0] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
            
    if(yf >= 0):
        for i in range(0,int(yf*1000),1):
            posicion[1] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    else:
        for i in range(0,int(yf*1000),-1):
            posicion[1] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    posicion[0] = xf
    posicion[1] = yf
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.0001)
    
        
    
    retCode = sim.simxSetJointTargetPosition(clientID, rotate_joint, (-30 - (angulo))*np.pi/180, sim.simx_opmode_oneshot)
    time.sleep(1)
    
       
    zf = -0.25
    for i in range(0,int(zf*1000),-1):
        posicion[2] = i/1000
        sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
        time.sleep(0.0001)
        
    posicion[2] = zf
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.0001)
    
    
    angulo = vision_objeto_unico(angulo)
    
    
    retCode = sim.simxSetJointTargetPosition(clientID, rotate_joint, (-30 - (angulo))*np.pi/180, sim.simx_opmode_oneshot)
    time.sleep(1)
    
    
    #posicion[2] = -0.38
    #sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    #time.sleep(2)
    
    
    zf = -0.38
    for i in range(int(-0.25*1000),int(zf*1000),-1):
        posicion[2] = i/1000
        sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
        time.sleep(0.0001)
        
    posicion[2] = zf
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.0001)
    
     
    dres = detect(gripper_sensor)
    holding = int(dres[0])
    res = gripper(1, gripper_attachpoint, int(dres[0]))
    time.sleep(3)
    
    
    zf = -0.18
    for i in range(int(-0.38*1000),int(zf*1000),+1):
        posicion[2] = i/1000
        sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
        time.sleep(0.001)
    
    posicion[2] = -0.18
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    
    
    
    
    return holding
    
    

In [1674]:
def volver_pos_original(x,y,z):
    #posicion[0] = x
    #posicion[1] = y
    #posicion[2] = z
    #sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    xa = posicion[0]
    ya = posicion[1]
    za = posicion[2]
    
    
    if(za >= 0):
        for i in range(int(za*1000),0,-1):
            posicion[2] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    else:
        for i in range(int(za*1000),0,1):
            posicion[2] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
            
            
    
    if(xa >= 0):
        for i in range(int(xa*1000),0,-1):
            posicion[0] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    else:
        for i in range(int(xa*1000),0,1):
            posicion[0] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    #posicion[0] = x
    #sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    #time.sleep(0.005)
    
    
    
    if(ya >= 0):
        for i in range(int(ya*1000),0,-1):
            posicion[1] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    else:
        for i in range(int(ya*1000),0,1):
            posicion[1] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    
    
    posicion[2] = z
    posicion[0] = x
    posicion[1] = y
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.0001)

    
    
            
    retCode = sim.simxSetJointTargetPosition(clientID, rotate_joint,-30*np.pi/180, sim.simx_opmode_oneshot)
    
    

In [1675]:
def desplazarse(direccion):
    if direccion == "ir":
        for i in range(0,2000,10):
            retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, i/2000, sim.simx_opmode_oneshot)
            time.sleep(0.000005)
    else:
        for i in range(2000,0,-10):
            retCode = sim.simxSetJointTargetPosition(clientID, traslate_joint, i/2000, sim.simx_opmode_oneshot)
            time.sleep(0.000005)
            

            
        
        
    

In [1676]:
def colocar(holding,x,y,columnas,tipo):
    if(tipo == 1):
        xf = (maximo2[0] - minimo2[0])/2
        yf = -((maximo2[1] - minimo2[1])/2) + columnas[1]*lego_altura
        columnas[1] = columnas[1] + 1
    else:
        xf = -(maximo2[0] - minimo2[0])/2
        yf = -((maximo2[1] - minimo2[1])/2) + columnas[1]*lego_altura
        columnas[0] = columnas[0] + 1
     
    if(xf >= 0):
        for i in range(0,int(xf*1000),1):
            posicion[0] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    else:
        for i in range(0,int(xf*1000),-1):
            posicion[0] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    
    if(yf >= 0):
        for i in range(0,int(yf*1000),1):
            posicion[1] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    else:
        for i in range(0,int(yf*1000),-1):
            posicion[1] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.0001)
    
    
    
    
    #sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    #time.sleep(1)
    zf = -0.38
    for i in range(0,int(zf*1000),-1):
            posicion[2] = i/1000
            sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
            time.sleep(0.05)
    posicion[0] = xf
    posicion[1] = yf
    posicion[2] = zf
    sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    time.sleep(0.0001)
    
    #sim.simxSetObjectPosition(clientID,target,home,posicion,sim.simx_opmode_oneshot)
    #time.sleep(2)
    res = gripper(0, gripper_attachpoint,holding)
    holding = ""
    time.sleep(3)
    return columnas
    

In [1677]:
#primera pieza 
#lo unico que cambia entre pieza y pieza es el hecho de poner 0 ---> pequeña  1----> grande
# buscar_pieza( 0   o   1)         \
#                                  O --->  esto es lo unico que hay qu8e cambiar dependiendo de lo que se busque, 0 o 1
# colocar (  ,  ,  ,  , 0  o  1)  /

#actualmente solo falta hacer bucle y ver que no falle nada si se intenta con una segunda pieza o incluso una tercera o quarta

In [1678]:
lista_piezas = [0,1,1,0,1] #recordar 0 son pequeñas y 1 son grandes

In [1679]:
for i in lista_piezas:
    posicion[0],posicion[1],angulo = buscar_pieza(i)
    holding = funcion_coger(posicion, angulo)
    volver_pos_original(origen_x,origen_y,origen_z)
    desplazarse("ir")
    columnas = colocar(holding,origen_x,origen_y,columnas,i)
    volver_pos_original(origen_x,origen_y,origen_z)
    desplazarse("volver")
    

x_f
-0.1485351439914666
160 176 53 57
x_f
0.1333007702487521
28 160 95 61
x_f
-0.13837889482965693
138 121 89 44
x_f
0.14218748826533556
41 89 62 60
x_f
-0.008886718016583472
94 12 75 99
handle objeto detectado:  196
distacia al objeto:  0.11720999881656077
x_f
0.10664061619900167
39 161 94 60
x_f
-0.16503904887940735
148 121 90 44
x_f
0.11552733421558514
52 89 61 60
x_f
-0.0368164032115601
105 12 75 99
handle objeto detectado:  167
distacia al objeto:  0.11808931849115067
x_f
0.09013671131106094
45 160 95 61
x_f
-0.18154295376734808
154 121 91 44
x_f
0.09902342932764441
59 89 60 60
handle objeto detectado:  166
distacia al objeto:  0.1170766800797307
x_f
-0.17646482918644324
152 121 91 44
x_f
0.10410155390854925
56 89 62 60
handle objeto detectado:  194
distacia al objeto:  0.116090859449172
x_f
-0.16503904887940735
148 121 90 44
handle objeto detectado:  164
distacia al objeto:  0.11907867366993613
